In [1]:
#see

!pip install neo4j

In [14]:
# Install a package using pip within a Jupyter Notebook
!pip install graphdatascience

In [3]:
import pandas as pd
import numpy as np

In [1]:
#read csv to dataframe
import pandas as pd
FILE = 'movies_export.csv'
df = pd.read_csv(FILE)

In [2]:
df.head()

,movieId,title,avgVote,releaseYear,tagline,genres,personType,personId,name,birthYear,deathYear,characters
0,189,Sin City: A Dame to Kill For,6.9,2014,There is no justice without sin.,Action:Crime:Drama:Thriller,ACTOR,2295,Mickey Rourke,1952,0,Marv
1,189,Sin City: A Dame to Kill For,6.9,2014,There is no justice without sin.,Action:Crime:Drama:Thriller,ACTOR,56731,Jessica Alba,1981,0,Nancy
2,189,Sin City: A Dame to Kill For,6.9,2014,There is no justice without sin.,Action:Crime:Drama:Thriller,ACTOR,16851,Josh Brolin,1968,0,Dwight
3,189,Sin City: A Dame to Kill For,6.9,2014,There is no justice without sin.,Action:Crime:Drama:Thriller,ACTOR,24045,Joseph Gordon-Levitt,1981,0,Johnny
4,189,Sin City: A Dame to Kill For,6.9,2014,There is no justice without sin.,Action:Crime:Drama:Thriller,ACTOR,5916,Rosario Dawson,1979,0,Gail


In [ ]:
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "{password}"
NEO4J_URI = "bolt://localhost:7687"

In [10]:
from graphdatascience import GraphDataScience

gds = GraphDataScience(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD) #,
    # aura_ds=False
)
gds.set_database('neo4j')

Unable to connect to the Neo4j DBMS. Trying again...
Unable to connect to the Neo4j DBMS. Trying again...


In [11]:
print(gds.version())

2.11.0


In [13]:


# gds.run_cypher('CREATE CONSTRAINT FOR (n:Movie) REQUIRE (n.title) IS UNIQUE')
# gds.run_cypher('CREATE CONSTRAINT FOR (n:Person) REQUIRE (n.name) IS UNIQUE')


In [113]:
#CREATE INDEX FOR (m:Movie) ON (m.released)
gds.run_cypher('CREATE INDEX FOR (m:Movie) ON (m.releaseYear)')

ClientError: {code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent index already exists, 'Index( id=6, name='index_65fd4c2a', type='RANGE', schema=(:Movie {releaseYear}), indexProvider='range-1.0' )'.}

In [14]:
movies_entries = df.to_dict(orient='records')

In [15]:
len(movies_entries)

64018

In [16]:
#print first dictionary entry
print(movies_entries[0:5])

#MATCH (n) DETACH DELETE n

[{'movieId': 189, 'title': 'Sin City: A Dame to Kill For', 'avgVote': 6.9, 'releaseYear': 2014, 'tagline': 'There is no justice without sin.', 'genres': 'Action:Crime:Drama:Thriller', 'personType': 'ACTOR', 'personId': 2295, 'name': 'Mickey Rourke', 'birthYear': 1952, 'deathYear': 0, 'characters': 'Marv'}, {'movieId': 189, 'title': 'Sin City: A Dame to Kill For', 'avgVote': 6.9, 'releaseYear': 2014, 'tagline': 'There is no justice without sin.', 'genres': 'Action:Crime:Drama:Thriller', 'personType': 'ACTOR', 'personId': 56731, 'name': 'Jessica Alba', 'birthYear': 1981, 'deathYear': 0, 'characters': 'Nancy'}, {'movieId': 189, 'title': 'Sin City: A Dame to Kill For', 'avgVote': 6.9, 'releaseYear': 2014, 'tagline': 'There is no justice without sin.', 'genres': 'Action:Crime:Drama:Thriller', 'personType': 'ACTOR', 'personId': 16851, 'name': 'Josh Brolin', 'birthYear': 1968, 'deathYear': 0, 'characters': 'Dwight'}, {'movieId': 189, 'title': 'Sin City: A Dame to Kill For', 'avgVote': 6.9, 'r

In [17]:
total = len(movies_entries)
count = 0

#first 200 rows
for r in movies_entries[:200]:
        gds.run_cypher('''
        UNWIND $records AS record
        MERGE (m:Movie {title: record.title})
        ON CREATE SET m.releaseYear = record.releaseYear, m.tagline = record.tagline
        MERGE (p:Person {name: record.name})
        ON CREATE SET p.birthYear = record.birthYear
        MERGE (p)-[r:ACTED_IN]->(m)
        RETURN count(m) as cnt
        ''', params={'records': [r]})
        count += 1
        print(f'loaded {count}')


loaded 1
loaded 2
loaded 3
loaded 4
loaded 5
loaded 6
loaded 7
loaded 8
loaded 9
loaded 10
loaded 11
loaded 12
loaded 13
loaded 14
loaded 15
loaded 16
loaded 17
loaded 18
loaded 19
loaded 20
loaded 21
loaded 22
loaded 23
loaded 24
loaded 25
loaded 26
loaded 27
loaded 28
loaded 29
loaded 30
loaded 31
loaded 32
loaded 33
loaded 34
loaded 35
loaded 36
loaded 37
loaded 38
loaded 39
loaded 40
loaded 41
loaded 42
loaded 43
loaded 44
loaded 45
loaded 46
loaded 47
loaded 48
loaded 49
loaded 50
loaded 51
loaded 52
loaded 53
loaded 54
loaded 55
loaded 56
loaded 57
loaded 58
loaded 59
loaded 60
loaded 61
loaded 62
loaded 63
loaded 64
loaded 65
loaded 66
loaded 67
loaded 68
loaded 69
loaded 70
loaded 71
loaded 72
loaded 73
loaded 74
loaded 75
loaded 76
loaded 77
loaded 78
loaded 79
loaded 80
loaded 81
loaded 82
loaded 83
loaded 84
loaded 85
loaded 86
loaded 87
loaded 88
loaded 89
loaded 90
loaded 91
loaded 92
loaded 93
loaded 94
loaded 95
loaded 96
loaded 97
loaded 98
loaded 99
loaded 100
loaded 1

In [18]:
# result = gds.run_cypher('MATCH (n) RETURN n')
result = gds.run_cypher('MATCH (people:Person) RETURN people.name LIMIT 100')
print(result)

           people.name
0         Keanu Reeves
1     Carrie-Anne Moss
2   Laurence Fishburne
3         Hugo Weaving
4       Andy Wachowski
..                 ...
95        Emile Hirsch
96        John Goodman
97      Susan Sarandon
98         Matthew Fox
99     Christina Ricci

[100 rows x 1 columns]


In [19]:
#MATCH (nineties:Movie) WHERE nineties.released >= 1990 AND nineties.released < 2000 RETURN nineties.title
result = gds.run_cypher("MATCH (teens:Movie) WHERE teens.releaseYear >= 2010 AND teens.releaseYear < 2020 RETURN teens.title")
print(result)

                          teens.title
0        Sin City: A Dame to Kill For
1             Guardians of the Galaxy
2                            Non-Stop
3                            Sex Tape
4   A Million Ways to Die in the West
5           Jack Ryan: Shadow Recruit
6                      Need for Speed
7                             Plastic
8                           The Giver
9                           Neighbors
10                           Godzilla
11                           Autómata
12                    Life after Beth
13                               7500
14                   Edge of Tomorrow
15                            Blended


## Verify if the Person node for Javier Bardem exists

In [20]:
#MATCH (javier:Person {name: 'Javier Bardem'})-[:ACTED_IN]->(javierMovies) RETURN javier,javierMovies
result = gds.run_cypher("MATCH (javier:Person {name: 'Javier Bardem'})-[:ACTED_IN]->(javierMovies) RETURN javier.name,javierMovies.title")
print(result)


     javier.name javierMovies.title
0  Javier Bardem           Autómata
